In [175]:
import sys
import subprocess

#import pkg_resources
#required = {'harmonypy','sklearn','scanpy','pandas', 'numpy', 'bbknn', 'scipy', 'matplotlib', 'seaborn' ,'scipy'}
#installed = {pkg.key for pkg in pkg_resources.working_set}
#missing = required - installed
#if missing:
#    print("Installing missing packages:" )
#    print(missing)
#    python = sys.executable
#    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
from bbknn import bbknn
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
import harmonypy as hm
from pathlib import Path
import ppscore as pps
import pathlib
import os

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=300, color_map='viridis')
plt.rcParams['figure.figsize'] = [15, 15]

In [176]:
run_name = 'Early_late_DEGS'
Object = '/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_data/A4_V7_YS_integrated_data_singlets_normalised_counts_for_MS_plotting_20211111_with_obsp.h5ad'
save_path = "/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/DEGS/"
pd_paths = "/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/sig_FC_early_late_neighbourhoods"

# Required: Name of first object
data1 = "_ys"
# Provide categorical to join between datasets
cat1 = "cell.labels"

In [161]:
import os
if not os.path.exists(save_loc):
    os.makedirs(save_loc)
adata = sc.read(Object)
adata.obs.columns

Index(['cell.labels', 'stage', 'lanes', 'sex', 'sort.ids', 'fetal.ids',
       'component', 'broad_cell.labels', 'sequencing.type', 'orig.dataset',
       'broad_cell_labels_organ', 'tissue', 'V6_old_indexes', 'n_counts'],
      dtype='object')

# Update adata metadata

In [162]:
obs = pd.read_csv('/nfs/team205/ig7/work_backups/backup_210306/projects/YS/resources/A2_V7_YS_main_metadata_161121.csv',index_col=0)
adata.obs = obs
adata.obs.index = adata.obs.index.str.replace('_1','')

In [167]:
adata.obs

,cell.labels,stage,lanes,sex,sort.ids,fetal.ids,component,broad_cell.labels,sequencing.type,orig.dataset,broad_cell_labels_organ,tissue,V6_old_indexes,V5_old_indexes
Wang_et_al_AAACCTGAGACCTTTG,Sinusoidal_EC,CS10,Wang_CS10,Male,Total,Wang_CS10,Membrane,Endothelium,TBC,Wang_et_al_Cell_Stem Cell_2021,Endothelium_ys,YS,Wang_et_al_AAACCTGAGACCTTTG_1,Cell_dat_Yolksac_Cell_dat_Yolksac_AAACCTGAGACC...
Wang_et_al_AAACCTGCACCTATCC,MEMP,CS10,Wang_CS10,Male,Total,Wang_CS10,Membrane,Progenitors,TBC,Wang_et_al_Cell_Stem Cell_2021,Progenitors_ys,YS,Wang_et_al_AAACCTGCACCTATCC_1,Cell_dat_Yolksac_Cell_dat_Yolksac_AAACCTGCACCT...
Wang_et_al_AAACCTGCACTTAAGC,HSPC_1,CS10,Wang_CS10,Male,Total,Wang_CS10,Membrane,Progenitors,TBC,Wang_et_al_Cell_Stem Cell_2021,Progenitors_ys,YS,Wang_et_al_AAACCTGCACTTAAGC_1,Cell_dat_Yolksac_Cell_dat_Yolksac_AAACCTGCACTT...
Wang_et_al_AAACCTGGTAACGACG,Erythroid,CS10,Wang_CS10,Male,Total,Wang_CS10,Membrane,Erythroid,TBC,Wang_et_al_Cell_Stem Cell_2021,Erythroid_ys,YS,Wang_et_al_AAACCTGGTAACGACG_1,Cell_dat_Yolksac_Cell_dat_Yolksac_AAACCTGGTAAC...
Wang_et_al_AAACCTGGTCAAAGAT,Erythroid,CS10,Wang_CS10,Male,Total,Wang_CS10,Membrane,Erythroid,TBC,Wang_et_al_Cell_Stem Cell_2021,Erythroid_ys,YS,Wang_et_al_AAACCTGGTCAAAGAT_1,Cell_dat_Yolksac_Cell_dat_Yolksac_AAACCTGGTCAA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGGTTCATCCGTGG,Erythroid,CS15,FCAImmP8096696,Male,CD45NAF,F80,Membrane,Erythroid,5GEX,De_novo,Erythroid_ys,YS,F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGGTTCATC...,F80_Yolksac_Hep_FCAImmP8096696_TTTGGTTCATCCGTGG
F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCAAGATGCCAG,Immature_EC,CS15,FCAImmP8096696,Male,CD45NAF,F80,Membrane,Endothelium,5GEX,De_novo,Endothelium_ys,YS,F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCAAGAT...,F80_Yolksac_Hep_FCAImmP8096696_TTTGTCAAGATGCCAG
F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCAGTCGAACAG,Immature_EC,CS15,FCAImmP8096696,Male,CD45NAF,F80,Membrane,Endothelium,5GEX,De_novo,Endothelium_ys,YS,F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCAGTCG...,F80_Yolksac_Hep_FCAImmP8096696_TTTGTCAGTCGAACAG
F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCATCATTCACT,Erythroid,CS15,FCAImmP8096696,Male,CD45NAF,F80,Membrane,Erythroid,5GEX,De_novo,Erythroid_ys,YS,F80_Yolksac_CD45NAF_FCAImmP8096696_TTTGTCATCAT...,F80_Yolksac_Hep_FCAImmP8096696_TTTGTCATCATTCACT


In [163]:
np.max(adata.X)

8.8560095

In [168]:
indexer = pd.read_csv(fpath,index_col=0)

In [146]:
indexer

,kth_distance,SpatialFDR,logFC,logCPM,F,PValue,FDR,nhood_anno,anno_group,nhood_Sig_FC,cell_nhood_Sig_FC
index_cell,,,,,,,,,,,
Wang_et_al_ATCTACTAGGGTCTCC,8.552673,0.002446,-2.966471,8.467364,15.567464,0.000080,0.002377,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_CACACCTTCGGCGCAT,14.000013,0.052103,-1.999833,8.395988,8.628561,0.003310,0.048546,Early_Erythroid,Erythroid,Late,Early_Erythroid_Late
Wang_et_al_CACACTCAGCAGCGTA,10.643909,0.002352,-3.065660,8.625258,15.671806,0.000075,0.002289,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_CGCTATCGTTCTCATT,10.082169,0.104067,-1.907349,8.679844,7.229602,0.007172,0.094728,Early_Erythroid,Erythroid,Late,Early_Erythroid_Late
Wang_et_al_CGGGTCAGTCGGGTCT,10.604790,0.075322,-2.046713,8.774702,7.894815,0.004958,0.069391,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_CGTTCTGAGCGATCCC,4.552075,0.009558,-2.951570,9.297908,12.363911,0.000438,0.009204,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_CTAGTGACACATTTCT,9.951945,0.000332,-3.389751,8.368503,20.266913,0.000007,0.000321,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_CTGTGCTAGGAATCGC,10.051715,0.005838,-2.716497,8.491235,13.506328,0.000238,0.005538,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early
Wang_et_al_GGAACTTAGTGTACCT,4.494466,0.004773,-3.240345,9.376059,14.015385,0.000181,0.004617,Early_Erythroid,Erythroid,Early,Early_Erythroid_Early


In [170]:
temp

View of AnnData object with n_obs × n_vars = 0 × 30330
    obs: 'cell.labels', 'stage', 'lanes', 'sex', 'sort.ids', 'fetal.ids', 'component', 'broad_cell.labels', 'sequencing.type', 'orig.dataset', 'broad_cell_labels_organ', 'tissue', 'V6_old_indexes', 'V5_old_indexes'
    var: 'n_cells', 'mt', 'total_counts', 'highly_variable', 'intersect', 'means', 'dispersions', 'dispersions_norm'
    uns: 'broad_cell.labels_colors', 'hvg', 'neighbors', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'
    obsp: 'connectivities', 'distances'

In [ ]:
concat_de = pd.DataFrame(index = list(range(0,30)))
rm_ls = list(adata.var[adata.var.index.str.startswith("MT-")].index) + list(adata.var[adata.var.index.str.startswith("RP11-")].index)
adata = adata[:,list(adata.var[~adata.var.index.isin(rm_ls)].index)]
paths = pd.DataFrame(pathlib.Path(pd_paths).glob("*early_late.csv"),columns = ['fpaths'])
for fpath in paths['fpaths']:
    enr_name = str(fpath).replace(pd_paths+'/','')
    enr_name = str(enr_name).replace('_sig_fc_early_late.csv','')
    print(enr_name)
    indexer = pd.read_csv(fpath,index_col=0)
    
    # make some fake neighbourhoods for computation
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Early'])]) < 2:
        indexer.loc[indexer['logFC']<np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Early'
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Late'])]) < 2:
        indexer.loc[indexer['logFC']>np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Late'

    indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]    
    # if len still <0 then ignore this data
    if min(indexer.groupby('nhood_Sig_FC').count().iloc[:,1]) >= 2 :
        indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]
        indexer['cell_nhood_Sig_FC'] = enr_name + "_" + indexer['nhood_Sig_FC'].astype(str) 
        temp = adata[adata.obs.index.isin(indexer['index_cell'])]
        indexer = indexer.set_index('index_cell')
        temp.obs['cell_nhood_Sig_FC'] = indexer['cell_nhood_Sig_FC']

        sc.tl.rank_genes_groups(temp, "cell_nhood_Sig_FC", method='wilcoxon',n_genes=30)
        result = temp.uns['rank_genes_groups']
        groups = result['names'].dtype.names
        DE = pd.DataFrame(
            {group + '_' + key[:1]: result[key][group]
            for group in groups for key in ['names', 'pvals','logfoldchanges']}).head(30)

        DE_name = (save_path + enr_name + '_' + 'DEGS.csv')
        DE.to_csv(DE_name)
        concat_de = pd.concat([concat_de, DE], axis=1)
concat_de.to_csv(save_path + 'all_concat_' + 'DEGS.csv')

# By Broad

In [178]:
concat_indexer = pd.DataFrame()
concat_indexer

""


In [182]:
indexer

,kth_distance,SpatialFDR,logFC,logCPM,F,PValue,FDR,nhood_anno,anno_group,nhood_Sig_FC,cell_nhood_Sig_FC
index_cell,,,,,,,,,,,
F32_Yolksac_Total_FCAImmP7316900_TCGAGGCAGATAGCAT,15.496283,0.662833,-0.049197,8.038840,0.506621,0.476606,0.674813,NK,Lymphoid,Early,NK_Early
F61_Yolksac_CD45P_FCAImmP7862089_ATCATGGTCGGCCGAT,19.260414,0.881586,0.386803,7.758015,0.077820,0.780274,0.891546,NK,Lymphoid,False,NK_False
F80_Yolksac_CD45P_FCAImmP8096693_GGGCATCTCAAGGCTT,15.376731,0.664650,1.680288,8.166450,0.500375,0.479337,0.676398,NK,Lymphoid,Late,NK_Late


In [186]:
concat_indexer = pd.DataFrame()
paths = pd.DataFrame(pathlib.Path(pd_paths).glob("*early_late.csv"),columns = ['fpaths'])
for fpath in paths['fpaths']:
    enr_name = str(fpath).replace(pd_paths+'/','')
    enr_name = str(enr_name).replace('_sig_fc_early_late.csv','')
    print(enr_name)
    indexer = pd.read_csv(fpath,index_col=0)
    # make some fake neighbourhoods for computation
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Early'])]) < 2:
        indexer.loc[indexer['logFC']<np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Early'
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Late'])]) < 2:
        indexer.loc[indexer['logFC']>np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Late'
        
    #indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]    
    
    # if len still <0 then ignore this data

    #indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]
    indexer['cell_nhood_Sig_FC'] = enr_name + "_" + indexer['nhood_Sig_FC'].astype(str) 
    indexer = indexer.set_index('index_cell')
    concat_indexer = pd.concat([concat_indexer, indexer], axis=0)
concat_indexer.to_csv('all_milo_outs.csv')

NK
HE
Endoderm
Monocyte
MOP
Monocyte_Macrophage
Early_Erythroid
Erythroid
LMPP
Pre_Macrophage
CMP
Mesothelium
Prolif_Sinusoidal_EC
Mono Mac pre DC2
MK
Sinusoidal_EC
Macrophage
MEMP
Prolif_AEC
Eo_Baso_Mast_precursor
VWF_EC
AEC
Mac DC2
Neutrophil_precursor
Immature_EC
Eo_Basophil
HSPC_2
Fibroblast
Microglia
HSPC_1
Cycling DC2
Lymphoid_progenitor
Promonocyte
Mast_cell
Pre DC2
Smooth_Muscle
pDC precursor
Mono Mac DC2


In [190]:
concat_indexer['broad_cell.labels'] = adata.obs['broad_cell.labels'][adata.obs.index.isin(concat_indexer.index)]
concat_indexer['broad_cell.labels']

index_cell
F32_Yolksac_Total_FCAImmP7316900_TCGAGGCAGATAGCAT       Lymphoid
F61_Yolksac_CD45P_FCAImmP7862089_ATCATGGTCGGCCGAT       Lymphoid
F80_Yolksac_CD45P_FCAImmP8096693_GGGCATCTCAAGGCTT       Lymphoid
Wang_et_al_CATGGCGTCGCCGTGA_2                        Endothelium
F61_Yolksac_CD45P_FCAImmP7862089_ATTGGTGGTTAGGGTG    Endothelium
                                                        ...     
F37_Yolksac_Total_FCAImmP7504911_TACACGAAGTGTGAAT             DC
F37_Yolksac_Total_FCAImmP7504913_AAACCTGCACCCATTC             DC
F37_Yolksac_Total_FCAImmP7504913_ACTGTCCGTCAATACC             DC
F37_Yolksac_Total_FCAImmP7504913_CCGTACTCAGGCTCAC             DC
F37_Yolksac_Total_FCAImmP7504913_CGTCCATGTCTGGAGA             DC
Name: broad_cell.labels, Length: 5192, dtype: object

In [191]:
run_name = 'Early_late_DEGS'
Object = '/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_data/A4_V7_YS_integrated_data_singlets_normalised_counts_for_MS_plotting_20211111_with_obsp.h5ad'
save_path = "/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/broad_DEGS/"
pd_paths = "/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/sig_FC_early_late_neighbourhoods"

In [192]:
import os
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [193]:
concat_indexer['broad_cell.labels'].unique()

array(['Lymphoid', 'Endothelium', 'Endoderm', 'Monocyte', 'Progenitors',
       'Erythroid', 'Macrophage', 'Mesothelium', 'DC', 'MK',
       'Granulocyte_precursors', 'Fibroblast', 'Microglia', 'Mast_cell',
       'Smooth_Muscle'], dtype=object)

In [197]:
concat_de = pd.DataFrame(index = list(range(0,30)))
rm_ls = list(adata.var[adata.var.index.str.startswith("MT-")].index) + list(adata.var[adata.var.index.str.startswith("RP11-")].index)
adata = adata[:,list(adata.var[~adata.var.index.isin(rm_ls)].index)]
for enr_name in concat_indexer['broad_cell.labels'].unique():
    indexer = concat_indexer[concat_indexer['broad_cell.labels'].isin([enr_name])]
    
    # make some fake neighbourhoods for computation
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Early'])]) < 2:
        indexer.loc[indexer['logFC']<np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Early'
    if len(indexer[indexer['nhood_Sig_FC'].isin(['Late'])]) < 2:
        indexer.loc[indexer['logFC']>np.median(indexer['logFC']),'nhood_Sig_FC'] = 'Late'

    indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]    
    # if len still <0 then ignore this data
    if min(indexer.groupby('nhood_Sig_FC').count().iloc[:,1]) >= 2 :
        indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]
        indexer['cell_nhood_Sig_FC'] = enr_name + "_" + indexer['nhood_Sig_FC'].astype(str) 
        temp = adata[adata.obs.index.isin(indexer.index)]
        temp.obs['cell_nhood_Sig_FC'] = indexer['cell_nhood_Sig_FC']

        sc.tl.rank_genes_groups(temp, "cell_nhood_Sig_FC", method='wilcoxon',n_genes=30)
        result = temp.uns['rank_genes_groups']
        groups = result['names'].dtype.names
        DE = pd.DataFrame(
            {group + '_' + key[:1]: result[key][group]
            for group in groups for key in ['names', 'pvals','logfoldchanges']}).head(30)

        DE_name = (save_path + enr_name + '_' + 'DEGS.csv')
        DE.to_csv(DE_name)
        concat_de = pd.concat([concat_de, DE], axis=1)
concat_de.to_csv(save_path + 'all_concat_' + 'DEGS.csv')

/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
Trying to set attribute `.obs` of view, copying.


ranking genes


/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cell.labels' as categorical
/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'stage' as categorical
/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and 

    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


Trying to set attribute `.obs` of view, copying.


...

    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


Trying to set attribute `.obs` of view, copying.


ranking genes


/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cell.labels' as categorical
/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'stage' as categorical
/home/jovyan/my-conda-envs/workhorse/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and 

    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


In [172]:
indexer

,kth_distance,SpatialFDR,logFC,logCPM,F,PValue,FDR,nhood_anno,anno_group,nhood_Sig_FC,cell_nhood_Sig_FC
index_cell,,,,,,,,,,,
F37_Yolksac_Total_FCAImmP7504911_ACGCCGATCGGAATCT,11.869955,0.717579,-0.068881,8.940992,0.378307,0.538512,0.728189,Mono Mac DC2,DC,Early,Mono Mac DC2_Early
F37_Yolksac_Total_FCAImmP7504911_TACACGAAGTGTGAAT,14.569967,0.529941,2.537404,8.596890,1.078200,0.355878,0.546663,Mono Mac DC2,DC,Late,Mono Mac DC2_Late
F37_Yolksac_Total_FCAImmP7504913_AAACCTGCACCCATTC,15.041358,0.509084,2.545712,8.656769,1.077311,0.336579,0.525254,Mono Mac DC2,DC,Late,Mono Mac DC2_Late
F37_Yolksac_Total_FCAImmP7504913_ACTGTCCGTCAATACC,14.136736,0.501954,-0.437478,8.844351,0.949766,0.329781,0.519485,Mono Mac DC2,DC,Early,Mono Mac DC2_Early
F37_Yolksac_Total_FCAImmP7504913_CCGTACTCAGGCTCAC,7.172330,0.711098,-0.081709,9.033150,0.394981,0.529694,0.721451,Mono Mac DC2,DC,Early,Mono Mac DC2_Early
F37_Yolksac_Total_FCAImmP7504913_CGTCCATGTCTGGAGA,14.994838,0.581750,2.298752,8.458149,0.899993,0.399337,0.595793,Mono Mac DC2,DC,Late,Mono Mac DC2_Late


In [ ]:
sc.tl.rank_genes_groups(temp, "corr_concat", method='wilcoxon',groups=['HSC_MPP_liver','HSC_fbm'],n_genes=500)
sc.pl.rank_genes_groups(temp, n_genes=50, sharey=False)

result = adata_deg.uns['rank_genes_groups']
groups = result['names'].dtype.names
    
DE = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals','logfoldchanges']}).head(500)

DE_name ="./DEGS_fbm_vs_fliver_HSC"
DE.to_csv(DE_name)

In [62]:
indexer = pd.read_csv('/nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/sig_FC_early_late_neighbourhoods/Erythroid_sig_fc_early_late.csv',index_col=0)

In [63]:
# make some fake neighbourhoods for computation
if len(indexer[indexer['nhood_Sig_FC'].isin(['Early'])]) == 0:
    indexer.loc[indexer['logFC']<0,'nhood_Sig_FC'] = 'Early'
if len(indexer[indexer['nhood_Sig_FC'].isin(['Late'])]) == 0:
    indexer.loc[indexer['logFC']>0,'nhood_Sig_FC'] = 'Late'

In [64]:
indexer

,index_cell,kth_distance,SpatialFDR,logFC,logCPM,F,PValue,FDR,nhood_anno,anno_group,nhood_Sig_FC
0,Wang_et_al_AAACGGGTCCTGTAGA,6.180536,0.000015,-4.497923,8.876404,29.070324,6.992447e-08,0.000013,Erythroid,Erythroid,Early
4,Wang_et_al_AACCGCGCATTCCTGC,6.063393,0.012904,-2.365669,8.334441,11.694368,6.270962e-04,0.012475,Erythroid,Erythroid,Early
6,Wang_et_al_AACTCCCCACGAAACG,5.929513,0.000024,-4.303628,8.979277,26.920462,2.123335e-07,0.000024,Erythroid,Erythroid,Early
10,Wang_et_al_AAGGAGCGTAGAGGAA,4.831554,0.000021,-4.835707,9.725339,27.396250,1.660201e-07,0.000021,Erythroid,Erythroid,Early
14,Wang_et_al_ACGATGTAGTGGTCCC,13.574813,0.017201,-2.288882,8.303049,11.065112,8.799269e-04,0.016525,Erythroid,Erythroid,Early
...,...,...,...,...,...,...,...,...,...,...,...
5174,F80_Yolksac_CD45NAF_FCAImmP8096696_TGAGCATTCAA...,3.916534,0.817299,1.241190,8.179853,0.172661,6.777579e-01,0.829347,Erythroid,Erythroid,False
5177,F80_Yolksac_CD45NAF_FCAImmP8096696_TGCCCATTCCG...,6.164710,0.324989,3.007260,7.802582,1.858095,1.728473e-01,0.359026,Erythroid,Erythroid,Late
5184,F80_Yolksac_CD45NAF_FCAImmP8096696_TTAGGCACATG...,6.121181,0.329905,-0.558679,7.755043,1.779517,1.822111e-01,0.364488,Erythroid,Erythroid,Early
5187,F80_Yolksac_CD45NAF_FCAImmP8096696_TTCGGTCAGGC...,5.476635,0.323653,3.418419,7.947849,2.277812,1.312394e-01,0.357108,Erythroid,Erythroid,Late


In [68]:
# make some fake neighbourhoods for computation
if len(indexer[indexer['nhood_Sig_FC'].isin(['Early'])]) < 2:
    indexer.loc[indexer['logFC']<0,'nhood_Sig_FC'] = 'Early'
if len(indexer[indexer['nhood_Sig_FC'].isin(['Late'])]) < 2:
    indexer.loc[indexer['logFC']>0,'nhood_Sig_FC'] = 'Late'
indexer = indexer[~indexer['nhood_Sig_FC'].isin(['False'])]
temp = adata[adata.obs.index.isin(indexer['index_cell'])]
temp

View of AnnData object with n_obs × n_vars = 1414 × 30330
    obs: 'cell.labels', 'stage', 'lanes', 'sex', 'sort.ids', 'fetal.ids', 'component', 'broad_cell.labels', 'sequencing.type', 'orig.dataset', 'broad_cell_labels_organ', 'tissue', 'V6_old_indexes', 'V5_old_indexes'
    var: 'n_cells', 'mt', 'total_counts', 'highly_variable', 'intersect', 'means', 'dispersions', 'dispersions_norm'
    uns: 'broad_cell.labels_colors', 'hvg', 'neighbors', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'
    obsp: 'connectivities', 'distances'

In [13]:
paths = pd.DataFrame(pathlib.Path(pd_paths).glob("*early_late.csv"),columns = ['fpaths'])
for fpath in paths['fpaths']:
    enr_name = str(gsea_fpath).replace(pd_paths+'/','')
    enr_name = str(enr_name).replace('.csv','')
    print(enr_name)
    gsea = pd.read_csv(gsea_fpath,index_col=0)

    gsea = gsea[gsea['Adjusted P-value'] < 0.05]
    gsea['g_list'] = 'NAN'
    gsea_dict = {}

,fpaths
0,/nfs/team205/ig7/work_backups/backup_210306/pr...
1,/nfs/team205/ig7/work_backups/backup_210306/pr...
2,/nfs/team205/ig7/work_backups/backup_210306/pr...
3,/nfs/team205/ig7/work_backups/backup_210306/pr...
4,/nfs/team205/ig7/work_backups/backup_210306/pr...
5,/nfs/team205/ig7/work_backups/backup_210306/pr...
6,/nfs/team205/ig7/work_backups/backup_210306/pr...
7,/nfs/team205/ig7/work_backups/backup_210306/pr...
8,/nfs/team205/ig7/work_backups/backup_210306/pr...
9,/nfs/team205/ig7/work_backups/backup_210306/pr...


In [ ]:
gsea_paths = pd.DataFrame(pathlib.Path(save_loc).glob("*res_score.csv"),columns = ['fpaths'])
for gsea_fpath in gsea_paths['fpaths']:
    enr_name = str(gsea_fpath).replace(save_loc+'/','')
    enr_name = str(enr_name).replace('.csv','')
    print(enr_name)
    gsea = pd.read_csv(gsea_fpath,index_col=0)

    gsea = gsea[gsea['Adjusted P-value'] < 0.05]
    gsea['g_list'] = 'NAN'
    gsea_dict = {}
    for z in gsea.index:
        term = (gsea['Term'][gsea.index==z].item())
        gsea_dict[term] = (gsea.loc[gsea.index == z ,'Genes'][z]).split(";")
        if len(gsea_dict[term]) >= min_term:
            gsea.at[z,'g_list'] = gsea_dict[term]
    gsea = gsea[~gsea['g_list'].isin(['NAN'])]
    unique_genes = list(np.unique(np.array([item for sublist in gsea['g_list'].values for item in sublist])))
    gseapy_matrix = pd.DataFrame(columns=gsea.Term.unique(), index=unique_genes)
    for col in gseapy_matrix.columns:
        for index in gseapy_matrix.index:
            if index in gsea_dict[col]:
                gseapy_matrix.at[index,col] = 1 # this is not a clean way to do this // rewrite
            else:
                gseapy_matrix.at[index,col] = 0 # this is not a clean way to do this // rewrite
    for i in gseapy_matrix.columns:
        gseapy_matrix[i] = gseapy_matrix[i].astype('int')

In [ ]:
nfs/team205/ig7/work_backups/backup_210306/projects/YS/YS_310122_notebooks/fig1_background/scMILO/sig_FC_early_late_neighbourhoods

In [8]:
adata.obs["lr_batch"]  = adata.obs["tissue"].astype(str) + '_' +  adata.obs['fetal.ids'].astype(str)
adata.obs['common_cat'] = adata.obs['cell.labels'][:]

# DE loop
- Read pd
- Slice data
- DE calc
- save output
- concat globals